In [ ]:
#import sys
#sys.path.append('/Volumes/Storage/DS/DL/exttorch/src/')

In [9]:
!pip install torch~=2.6.0 'torch_xla[tpu]~=2.6.0' \
  -f https://storage.googleapis.com/libtpu-releases/index.html \
  -f https://storage.googleapis.com/libtpu-wheels/index.html
!pip install git+http://github.com/denis-spe/exttorch
!pip install columnar

Looking in links: https://storage.googleapis.com/libtpu-releases/index.html, https://storage.googleapis.com/libtpu-wheels/index.html
  Cloning http://github.com/denis-spe/exttorch to /tmp/pip-req-build-xpntud5w
  Running command git clone --filter=blob:none --quiet http://github.com/denis-spe/exttorch /tmp/pip-req-build-xpntud5w
  Resolved http://github.com/denis-spe/exttorch to commit 0d2d15f91720235860a50c4f14450229a438ff77
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [15]:
from sklearn.datasets import load_iris
import torch
from torch import nn
from torch.optim import Adam
from exttorch.accelerator import TpuScope
from exttorch.models import Sequential
from exttorch.callbacks import EarlyStopping
from exttorch.hyperparameter import HyperParameters
from exttorch import metrics
from exttorch import tuner
dir(metrics)

['Accuracy',
 'Auc',
 'F1Score',
 'Jaccard',
 'MatthewsCorrcoef',
 'MeanAbsoluteError',
 'MeanSquaredError',
 'Metric',
 'Precision',
 'R2',
 'Recall',
 'TopKAccuracy',
 'ZeroOneLoss',
 '__abc__',
 '__abs__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__']

In [16]:
early_stopping = EarlyStopping(3)

In [11]:
i_x, i_y = load_iris(return_X_y=True)

def tuned_model(hp: HyperParameters):
    features = hp.Choice('features', [128, 256, 512, 1062])
    h_features = hp.Int('h_features', 8, 1062, step=16)
    lr = hp.Float('lr', 0.0001, 0.001)

    with TpuScope():
        if hp.Boolean('deep_learning'):
            model = Sequential([
                nn.Linear(4, features),
                nn.Linear(features, h_features),
                nn.Linear(h_features, 3)])
        else:
            model = Sequential([
                nn.Linear(4, features),
                nn.Linear(features, 3)])

        model.compile(
             loss = nn.CrossEntropyLoss(),
             optimizer = Adam(model.parameters(), lr=lr),
             metrics = ["accuracy"]
        )

    return model

# Initialize the random search
random_search = tuner.RandomSearchTune(
                    tuned_model,
                    random_state=42,
                    objective = 'val_loss',
                    iterations=2
                )

# Search the parameters
random_search.search(
                i_x, i_y,
                epochs=5,
                #generator=torch.Generator().manual_seed(42),
                validation_data = (i_x, i_y)
            )

Iteration-2/2
Summary
| Previous val_loss: 2.7685
| Best val_loss so far: 2.7685

|             |                |            |
| Param Names | Previous param | Best param |
|===========================================|
|features     |512             |128         |
|             |                |            |
|h_features   |824             |8           |
|             |                |            |
|lr           |0.00061         |0.0001      |
|             |                |            |
|deep_learning|False           |False       |
|             |                |            |


 ---------------------------------------- 

Epoch 1/5
151/150 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 1.1973 - accuracy: 0.3333 - val_loss: 1.1979 - val_accuracy: 0.3333
Epoch 2/5
151/150 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 1.1973 - accuracy: 0.3333 - val_loss: 1.1979 - val_accuracy: 0.3333
Epoch 3/5
151/150 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 1.1973 - accuracy: 0.3333 - val_loss: 1.1979 - val

In [12]:
random_search.best_score

Best Score
val_loss: 1.1979


In [22]:
def tuned_model(hp: HyperParameters):
    features = hp.Choice('features', [128, 256])
    h_features = hp.Int('h_features', 8, 12, step=1)

    if hp.Boolean('deep_learning'):
        model = Sequential([
            nn.Linear(4, features),
            nn.LayerNorm(features),
            nn.ReLU(),
            nn.Linear(features, h_features),
            nn.LayerNorm(h_features),
            nn.Linear(h_features, 3)])
    else:
        model = Sequential([
        nn.Linear(4, features),
        nn.ReLU(),
        nn.Linear(features, 3)])

    model.compile(
        loss = nn.CrossEntropyLoss(),
        optimizer = Adam(model.parameters()),
        metrics = ["accuracy"]
    )

    return model

# Initialize the random search
random_search = tuner.GridSearchTune(
                    tuned_model,
                    objective = 'val_loss'
                )

# Search the parameters
random_search.search(
                i_x, i_y,
                epochs=100000,
                validation_data = (i_x, i_y),
                callbacks=[early_stopping]
            )

Iteration-16/16
Summary
| Previous val_loss: 0.54071
| Best val_loss so far: 0.50286

|             |                |            |
| Param Names | Previous param | Best param |
|===========================================|
|features     |256             |128         |
|             |                |            |
|h_features   |11              |10          |
|             |                |            |
|deep_learning|False           |False       |
|             |                |            |


 ---------------------------------------- 

Epoch 1/100000
151/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0984 - accuracy: 0.8333 - val_loss: 5.9256 - val_accuracy: 0.3333
Epoch 2/100000
151/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0661 - accuracy: 0.7600 - val_loss: 3.1871 - val_accuracy: 0.3333
Epoch 3/100000
151/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6900 - accuracy: 0.8000 - val_loss: 2.3194 - val_accuracy: 0.6667
Epoch 4/100000
151/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - 

In [23]:
random_search.best_scores

Overall Score
accuracy: 0.92264
val_accuracy: 0.81167
loss: 0.20482
val_loss: 0.50286
